In [4]:
import pandas as pd
from tqdm import tqdm

DATA_DIR = 'data/'

In [5]:
import pytorch_utils
import importlib
importlib.reload(pytorch_utils)

dataset = pytorch_utils.EntityDataset(device='mps')

[PyTorch-Utils]: Loading all-MiniLM-L6-v2 model & Generating sentence embeddings of Train set...


100%|██████████| 7375/7375 [00:01<00:00, 4924.67it/s]


Batches:   0%|          | 0/231 [00:00<?, ?it/s]

[PyTorch-Utils]: Now generating entity embeddings...


Batches:   0%|          | 0/572 [00:00<?, ?it/s]

[PyTorch-Utils]: Generating DataFrame for candidate generation...
[PyTorch-Utils]: Generating Embedding for candidate descriptions...


In [10]:
candidate_dict = {}
for i in tqdm(range(len(dataset))):
    try:
        full_mention = dataset.entity_df.iloc[i]['full_mention'].strip().lower()
        candidate_dict[full_mention] = dataset.at_count_df.loc[[full_mention]].head(5)['target_item_id'].to_list()
    except:
        pass

 61%|██████    | 11167/18288 [22:02<14:09,  8.38it/s]

In [ ]:
# save 
import pickle
with open(DATA_DIR + 'candidate_dict.pkl', 'wb') as f:
    pickle.dump(candidate_dict, f)

In [7]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4, collate_fn=lambda x: pytorch_utils.EntityDataset.collate_fn(x, device='cpu'))

In [8]:
for batch in tqdm(dataloader):
    pass

  0%|          | 0/572 [00:00<?, ?it/s]


RuntimeError: _share_filename_: only available on CPU

In [ ]:
import helper

at_count_df = helper.get_anchor_info(f'{EXTRA_DATA_DIR}link_annotated_text.jsonl', wiki_items_joined)

at_count_df

,en_label,en_description,normalized_anchor_text,target_page_id,target_item_id,target_page_title,target_page_views,anchor_target_count,p_anchor_given_target,p_target_given_anchor
0,United States of America,federal republic in North America,united states,3434750,30,United States,490557.0,144382,0.591523,0.954510
1,United States of America,federal republic in North America,american,3434750,30,United States,490557.0,68731,0.281586,0.798112
2,United States of America,federal republic in North America,usa,3434750,30,United States,490557.0,8361,0.034254,0.853860
3,United States of America,federal republic in North America,u.s.,3434750,30,United States,490557.0,7322,0.029998,0.957374
4,United States of America,federal republic in North America,us,3434750,30,United States,490557.0,5196,0.021288,0.803588
...,...,...,...,...,...,...,...,...,...,...
5619294,The Whole World is a Stage,1967 song performed by The Fantastic Four,the whole world is a stage,48600253,22043882,The Whole World is a Stage,38.0,2,1.000000,1.000000
5619295,2013 Franken Challenge – Doubles,NaN,●,39583307,13425873,2013 Franken Challenge – Doubles,9.0,2,1.000000,0.001946
5619296,"Edward Jessup, Jr.",Canadian politician,"edward jessup, jr.",4902036,5343763,Edward Jessup Jr.,11.0,2,1.000000,1.000000
5619297,Imraan Mohammad,English cricketer,imraan mohammad,29541591,6008103,Imraan Mohammad,21.0,2,1.000000,1.000000


In [ ]:
from helper import AnchorTargetStats, text_normalizer

anchor_target_counts = AnchorTargetStats(
    at_count_df, text_normalizer
)

In [ ]:
candidates_df = anchor_target_counts.get_disambiguation_candidates_from_text(
    "CBI"
).sort_values("p_target_given_anchor", ascending=False).head(5)

candidates = candidates_df['target_item_id']

In [ ]:
candidates_df = anchor_target_counts.get_disambiguation_candidates_from_text(
    "Williams"
).sort_values("p_target_given_anchor", ascending=False).head(15)

candidates_df

,en_label,en_description,normalized_anchor_text,target_page_id,target_item_id,target_page_title,target_page_views,anchor_target_count,p_anchor_given_target,p_target_given_anchor
111297,Williams Grand Prix Engineering,British Formula One motor racing team and cons...,williams,34104,171337,Williams Grand Prix Engineering,16423.0,1166,0.791045,0.466027
794605,WMS Industries,American gaming company and manufacturer of pi...,williams,870373,1403157,WMS Industries,1509.0,95,0.437788,0.037970
939743,Kelly Williams,American basketball player,williams,11929138,6386338,Kelly Williams,412.0,73,0.510490,0.029177
75678,Williams College,liberal arts college in Massachusetts,williams,34105,49166,Williams College,10181.0,71,0.039379,0.028377
976872,Elizabeth Williams,American basketball player,williams,46471671,20710856,Elizabeth Williams (basketball),392.0,69,0.841463,0.027578
154349,Deron Williams,American basketball player,williams,2127805,298496,Deron Williams,6549.0,62,0.069042,0.024780
1093712,Woody Williams,American baseball pitcher,williams,3261666,8033710,Woody Williams,266.0,57,0.527778,0.022782
1126267,Courtney M. Williams,American basketball player,williams,50179847,23884729,Courtney M. Williams,219.0,54,0.857143,0.021583
1016825,Williams,"city in Coconino County, Arizona, United States",williams,106603,79864,"Williams, Arizona",1502.0,54,0.457627,0.021583
396905,Lou Williams,American basketball player,williams,2447393,1354036,Lou Williams,23958.0,45,0.097614,0.017986


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

window_size = 4  # Example window size

def extract_context_window(row, doc_tokens, window_size):
    # Find the position of the entity in the all_tokens list
    position = doc_tokens.index(row['token'])

    # Calculate start and end positions of the window
    start = max(0, position - window_size)
    end = min(len(all_tokens), position + window_size + 1)
    # Extract the window of tokens
    window_tokens = doc_tokens[start:end]
    # add padding to the window if necessary
    if position < window_size: # if the entity is in the first few tokens
        window_tokens = ['<PAD>'] * (window_size - position) + window_tokens
    if len(all_tokens) - position < window_size: # if the entity is in the last few tokens
        window_tokens = window_tokens + ['<PAD>'] * (window_size - (len(all_tokens) - position))
    
    # Convert the window of tokens to a string
    return ' '.join(window_tokens)

# Assuming 'all_tokens' is a list of all tokens in the document
sample_entities['context_window'] = sample_entities.progress_apply(lambda row: extract_context_window(row, all_tokens[row['doc_id'] - 1], window_size), axis=1)

# Then apply TF-IDF Vectorizer to these context windows
tfidf_vectorizer = TfidfVectorizer()
contextual_features = tfidf_vectorizer.fit_transform(sample_entities['context_window'])

100%|██████████| 34043/34043 [00:00<00:00, 224640.89it/s]


In [ ]:
contextual_features.shape

(34043, 12917)

In [ ]:
# Semantic Features: Calculating similarity between entity mention and Wikidata labels/descriptions
# For demonstration, we'll use a simple method to calculate a similarity score
def calculate_similarity(text, candidates):
    if candidates.empty:
        return np.nan
    tfidf_candidates = tfidf_vectorizer.transform(candidates['en_label'].dropna())
    similarity = (tfidf_candidates * tfidf_vectorizer.transform([text]).T).toarray()
    return np.nanmax(similarity) if similarity.size > 0 else np.nan

sample_entities['semantic_similarity'] = sample_entities.apply(lambda x: calculate_similarity(x['full_mention'], entity_candidates.get(x['full_mention'], {}).get('candidates', pd.DataFrame())), axis=1)

# Relational Features: For simplicity, we'll count the number of relations in statements.csv for each entity
# This is a placeholder for more complex relational features that could be developed
def count_relations(item_id, statements):
    return len(statements[(statements['source_item_id'] == item_id) | (statements['target_item_id'] == item_id)])

sample_entities['relation_count'] = sample_entities.apply(lambda x: count_relations(entity_candidates.get(x['full_mention'], {}).get('candidates', pd.DataFrame()).iloc[0]['item_id'], statements) if not entity_candidates.get(x['full_mention'], {}).get('candidates', pd.DataFrame()).empty else 0, axis=1)

# Displaying the generated features for the sample entities
sample_entities[['full_mention', 'semantic_similarity', 'relation_count']]

In [ ]:
# Correcting the calculation of 'relation_count'
# This function counts the number of relations in statements.csv for each entity
def count_relations(item_id, statements):
    return len(statements[(statements['source_item_id'] == item_id) | (statements['target_item_id'] == item_id)])

# Applying the function to calculate relation count for sample entities
# We need to ensure that we have a valid item_id from the candidates for each entity
def get_first_valid_item_id(candidates):
    if not candidates.empty:
        valid_item_ids = candidates['item_id'].dropna()
        return valid_item_ids.iloc[0] if not valid_item_ids.empty else None
    return None

sample_train_entities['relation_count'] = sample_train_entities.apply(lambda x: count_relations(get_first_valid_item_id(entity_candidates.get(x['full_mention'], {}).get('candidates', pd.DataFrame())), statements), axis=1)

# Displaying the updated features for the sample entities
sample_train_entities[['full_mention', 'semantic_similarity', 'relation_count']]


KeyError: "['semantic_similarity'] not in index"